In [2]:
import pathlib
import os
import sys
import boto3

CURRENT_DIR = pathlib.Path('.').resolve().parent

# Adicione o diretório ao sys.path
sys.path.append(str(CURRENT_DIR))

from config import config

import pandas as pd
from datetime import datetime
import yfinance as yf

# data parameters
start_date = datetime(2022,1,1)
end_date = datetime(2024,1,1)

# getting the data
df_raw = yf.download('AAPL', start=start_date, end=end_date)
df_raw.reset_index(inplace=True)

display(df_raw)

[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-01-03,177.830002,182.880005,177.710007,182.009995,179.724533,104487900
1,2022-01-04,182.630005,182.940002,179.119995,179.699997,177.443573,99310400
2,2022-01-05,179.610001,180.169998,174.639999,174.919998,172.723572,94537600
3,2022-01-06,172.699997,175.300003,171.639999,172.000000,169.840271,96904000
4,2022-01-07,172.889999,174.139999,171.029999,172.169998,170.008133,86709100
...,...,...,...,...,...,...,...
496,2023-12-22,195.179993,195.410004,192.970001,193.600006,193.353287,37122800
497,2023-12-26,193.610001,193.889999,192.830002,193.050003,192.803986,28919300
498,2023-12-27,192.490005,193.500000,191.089996,193.149994,192.903839,48087700
499,2023-12-28,194.139999,194.660004,193.169998,193.580002,193.333298,34049900


# Transform Data

## Drop Columns

Data: A coluna "Date" geralmente não é usada como uma feature nos modelos de previsão financeira, pois a ordem temporal já é capturada pela sequência das linhas do dataframe.

Adj Close: A coluna "Adj Close" representa o preço de fechamento ajustado de uma ação, levando em consideração eventos como dividendos, divisões de ações, etc. Não vamos utilizar o preço de fechamento ajustado para a análise específica. Iremos descartar essa coluna para reduzir a dimensionalidade dos dados ou simplificar a análise.

In [3]:
df_transform = df_raw.copy()
df_transform.drop(columns=['Date'], inplace=True)
df_transform.drop(columns=['Adj Close'], inplace=True)

A estratégia abaixo cria um conjunto de dados onde cada linha contém um conjunto de features e um target (o valor da primeira coluna no dia seguinte). Como o problema proposto é previsão de séries temporais, usamos dados históricos para prever o próximo valor em uma série temporal. Neste caso, estamos utilizando a abertura do dia seguinte como target do dia anterior.

## Prepare Dataset

In [4]:
# Todas as linhas, exceto a última.
features = df_transform.iloc[:-1, :]

# Todas as linhas, exceto a primeira.
# Dessa forma, conseguimos pegar o fechamento desse dia.
targets = df_transform.iloc[1:, 0]

# Adiciona os targets como uma nova coluna em features, na primeira posição.
features.insert(0, 'Target', targets.values)

df_final = features.copy()
df_final = df_final.sample(frac=1, random_state=config.SEED)

## Split Data

In [5]:
# Número de linhas para treinamento (80%)
train_size = int(len(df_final) * 0.8)

# DataFrame de treinamento
train_data = df_final.iloc[:train_size]

# DataFrame de teste
test_data = df_final.iloc[train_size:]

print('Train shape', len(train_data))
print('Test shape', len(test_data))

Train shape 400
Test shape 100


# Load to S3 Bucket

In [6]:
train_data.to_csv(config.TRAIN_CSV_PATH, index=False, header=False)
test_data.to_csv(config.TEST_CSV_PATH, index=False, header=False)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:273: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)
